In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import pywt
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load_data
sampling_rate = 100  # Hz
duration = 10

data = np.load('/content/drive/MyDrive/Data/simu_3000_0.1_90_180.npy')
data_clean = np.load('/content/drive/MyDrive/Data/simu_3000_0_90_180.npy')
data_clean_NoRR = np.load('/content/drive/MyDrive/Data/simu_3000_0_90_180_NoRR.npy')


labels, signals =  data[:, -6:], data[:, :1000]
signals_clean =  data_clean[:, :1000]
signals_clean_NoRR =  data_clean_NoRR[:, :1000]

In [1]:
from scipy.signal import hilbert, savgol_filter

def standize(data):
    return (data - np.mean(data))/np.std(data)

def wavelet_decomposition(data, wave, Fs = None, n_decomposition = None):

    """
    n_decomposition:
      小波分解的层数或次数。每一次分解都会产生一组近似系数和一组细节系数。
      n_decomposition 决定了要将信号分解成多少个这样的组合。每增加一层分
      解，会得到更多的近似系数和细节系数。

    """
    a = data
    w = wave

    ca = []  # 存储近似系数
    cd = []  # 存储细节系数
    rec_a = []  # 存储重构的近似系数
    rec_d = []  # 存储重构的细节系数
    freq_range = []  # 存储频率范围

    for i in range(n_decomposition):
        if i == 0:
            freq_range.append(Fs/2)
        freq_range.append(Fs/2/(2** (i+1)))

        # 使用小波变换对数据进行分解
        (a, d) = pywt.dwt(a, w)
        ca.append(a)
        cd.append(d)

    for i, coeff in enumerate(ca):
        # 构造近似系数的系数列表，用于重构
        coeff_list = [coeff, None] + [None] * i
        rec = pywt.waverec(coeff_list, w)
        rec_a.append(rec)

    for i, coeff in enumerate(cd):
        # 构造细节系数的系数列表，用于重构
        coeff_list = [None, coeff] + [None] * i
        rec_d.append(pywt.waverec(coeff_list, w))

    return rec_a, rec_d

def wavelet_denoise(signal, Fs, n_decomposition):
    x = standize(signal)
    rec_a, rec_d = wavelet_decomposition(data = x, wave = 'db4', Fs = Fs, n_decomposition = n_decomposition)
    # 选择最短的细节系数作为去噪后的信号长度
    min_len = min(len(rec_d[-1]), len(rec_d[-2]), len(rec_d[-3]), len(rec_d[-4]))
    # 合并多个细节系数作为去噪后的信号
    denoised_sig = rec_d[-1][:min_len] + rec_d[-2][:min_len] + rec_d[-4][:min_len] + rec_d[-3][:min_len]
    return denoised_sig

